In [3]:
from pruners.pruning_methods import L1Unstructured, RandUnstructured
import torch
from evaluations.evaluator import Evaluator
from tqdm import tqdm
from transformers import AutoTokenizer, BertForSequenceClassification
from explainers.explanation_methods import SHAP, LIME
import shap
import torch
from copy import deepcopy

device = torch.device('cpu')

/Users/camilopinilla/anaconda3/envs/cs2822/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:


tokenizer = AutoTokenizer.from_pretrained(
    "textattack/bert-base-uncased-yelp-polarity")
model = BertForSequenceClassification.from_pretrained(
    "textattack/bert-base-uncased-yelp-polarity")
print('model loaded')

input = 'Hello, my dog is so terribly ugly'
tokenized_input = tokenizer(input, return_tensors="pt")
label = torch.tensor([0]).unsqueeze(0)  # Batch size 1

pruned_model = deepcopy(model)
unstructured_model = deepcopy(model)

l1_paras_to_prune = []
random_paras_to_prune = []


for layer in pruned_model.bert.encoder.layer:
    # Attention weights (query, key, value, and output projection)
    l1_paras_to_prune.append((layer.attention.self.query, 'weight'))
    l1_paras_to_prune.append((layer.attention.self.key, 'weight'))
    l1_paras_to_prune.append((layer.attention.self.value, 'weight'))
    l1_paras_to_prune.append((layer.attention.output.dense, 'weight'))
    
    # Intermediate dense layer
    l1_paras_to_prune.append((layer.intermediate.dense, 'weight'))
    
    # Output dense layer
    l1_paras_to_prune.append((layer.output.dense, 'weight'))

for layer in unstructured_model.bert.encoder.layer:
    # Attention weights (query, key, value, and output projection)
    random_paras_to_prune.append((layer.attention.self.query, 'weight'))
    random_paras_to_prune.append((layer.attention.self.key, 'weight'))
    random_paras_to_prune.append((layer.attention.self.value, 'weight'))
    random_paras_to_prune.append((layer.attention.output.dense, 'weight'))
    
    # Intermediate dense layer
    random_paras_to_prune.append((layer.intermediate.dense, 'weight'))
    
    # Output dense layer
    random_paras_to_prune.append((layer.output.dense, 'weight'))

# Initialize Pruner instance
Bert_pruner = Pruner(pruned_model)
Unstructured_pruner = Pruner(unstructured_model)

# Initialize PruningMethod instances
L1_prune_Bert = PruningMethod(type = "L1Unstructured", paras_to_prune = l1_paras_to_prune, percentage = 0.2, mask = None)
Unstructured_prune = PruningMethod(type='RandomUnstructured', paras_to_prune= random_paras_to_prune, percentage=0.2, mask=None)

# Prune the model
Bert_pruner.prune(L1_prune_Bert)
L1_prune_Bert.remover()

Unstructured_pruner.prune(Unstructured_prune)
Unstructured_prune.remover()

pruned_shapExplainer = SHAP(pruned_model, tokenizer, device=device)
unstructured_shapExplainer = SHAP(unstructured_model, tokenizer, device=device)
normal_shapExplainer = SHAP(model, tokenizer, device=device)

# pruned_explanation = pruned_shapExplainer.explain(input)
# normal_explanation = normal_shapExplainer.explain(input)

unstructured_evaluator = Evaluator(unstructured_shapExplainer)
pruned_evaluator = Evaluator(pruned_shapExplainer)
normal_evaluator = Evaluator(normal_shapExplainer)

inputs = ['Camilo CANNOT CODE FOR HIS LIFE. I DONT LIKE HIM!!!!',
          'David is GREAT at soccer. Can recommend <thumbs up>!',
          'Joey is joey. I feel very neutrally about him',
          'Finale is the best professor Harvard has EVER had. Would recommend!',
          'I AM GOING TO SCREAMMMMMMMMMMM AHHHHHHHHHHHH',
          'Paula and Hiwot are great TFs!']

for input in inputs:
    print(f'Evaluated on {input}')
    print(f'Normal infedility: {normal_evaluator.get_local_infidelity(input):4f}')
    print(f'Unstructured Pruning Infidelity: {unstructured_evaluator.get_local_infidelity(input):4f}')
    print(f'L1 Pruned infidelity: {pruned_evaluator.get_local_infidelity(input):4f}')
    print('*' * 80)

model loaded


NameError: name 'Pruner' is not defined

In [18]:
def eval_suite(model, tokenizer, inputs, prune_ptg):
    # Init models and grab model params to prune
    # TODO: Refactor this shi
    model.to(device)
    randunstrctured_model = deepcopy(model).to(device)
    l1unstructured_model = deepcopy(model).to(device)

    randunstruct_params = [param for name, param in randunstrctured_model.named_parameters() if 'embeddings' not in name]
    l1unstruct_params = [param for name, param in l1unstructured_model.named_parameters() if 'embeddings' not in name]

    # Initialize pruners and make pruned models
    print('Pruning models...')
    unpruned_model = model
    RandUnstructured(randunstrctured_model, 
                    randunstruct_params,
                    prune_ptg)
    L1Unstructured(l1unstructured_model, 
                    l1unstruct_params,
                    prune_ptg)
    

   # Init explainers
    shap_randunstruct, lime_randunstruct = SHAP(randunstrctured_model, tokenizer, device),\
                                           LIME(randunstrctured_model, tokenizer, device)
    shap_l1unstruct, lime_l1unstruct = SHAP(l1unstructured_model, tokenizer, device),\
                                       LIME(l1unstructured_model, tokenizer, device)
    shap_unpruned, lime_unpruned = SHAP(unpruned_model, tokenizer, device),\
                                   LIME(unpruned_model, tokenizer, device)
    
    # Init evaluators
    randunstruct_evaluators = {'shap': Evaluator(shap_randunstruct)}
                               #'lime': Evaluator(lime_randunstruct)}
    l1unstruct_evaluators = {'shap': Evaluator(shap_l1unstruct)}
                            # 'lime': Evaluator(lime_l1unstruct)}
    unrpuned_evaluators = {'shap': Evaluator(shap_unpruned)}
                           #'lime': Evaluator(lime_unpruned)}

    # Janky for now but hang with me
    infidelities_ = {'unpruned': unrpuned_evaluators,
                    'l1unstruct': l1unstruct_evaluators,
                    'randunstruct': randunstruct_evaluators}
    
    # Run evaluations, storing in dictionary of 
    # {prune_method: 
    #   {explanation_method: infidelity}
    # }
    print('Evaluating explanations...')
    infidelities = {}
    for input in tqdm(inputs, desc='Evaluating', unit='input'):
        for prune_method, evaluator_set in infidelities_.items():
            # Initialize prune_method in infidelities if not present
            if prune_method not in infidelities:
                infidelities[prune_method] = {}  # Initialize prune_method dict
            
            for expla_method, evaluator in evaluator_set.items():
                # Initialize expla_method as a list if not present
                if expla_method not in infidelities[prune_method]:
                    infidelities[prune_method][expla_method] = []
                
                # Append the result of get_local_infidelity to the list
                infidelities[prune_method][expla_method].append(evaluator.get_local_infidelity(input))

    return infidelities

In [19]:
device = torch.device('mps')
tokenizer = AutoTokenizer.from_pretrained(
    "textattack/bert-base-uncased-yelp-polarity")
model = BertForSequenceClassification.from_pretrained(
    "textattack/bert-base-uncased-yelp-polarity")
model = model.to(device)
# model_params = [param for name, param in model.named_parameters() if 'embeddings' not in name]

# l1_model = deepcopy(model).to(device)
# l1_params = [param for name, param in l1_model.named_parameters() if 'embeddings' not in name]

# unstruct_model = deepcopy(model).to(device)
# unstruct_params = [param for name, param in unstruct_model.named_parameters() if 'embeddings' not in name]

# L1Unstructured(l1_model, l1_params, .2)
# RandUnstructured(unstruct_model, unstruct_params, .2)

# l1_shap, l1_lime = SHAP(l1_model, tokenizer, device), LIME(l1_model, tokenizer, device)
# l1_shap_evaluator, l1_lime_evaluator = Evaluator(l1_shap), Evaluator(l1_lime)



inputs = ['Camilo CANNOT CODE FOR HIS LIFE. I DONT LIKE HIM!!!!',
          'David is GREAT at soccer. Can recommend <thumbs up>!',
          'Joey is joey. I feel very neutrally about him',
          'Finale is the best professor Harvard has EVER had. Would recommend!',
          'I AM GOING TO SCREAMMMMMMMMMMM AHHHHHHHHHHHH',
          'Paula and Hiwot are great TFs!']

# l1_shap_evaluator.get_local_infidelity(inputs[0])
# l1_lime_evaluator.get_local_infidelity(inputs[0])
infidelities = eval_suite(model, tokenizer, inputs, .20)

Pruning models...
Evaluating explanations...


Evaluating: 100%|██████████| 6/6 [01:07<00:00, 11.22s/input]


In [20]:
infidelities

{'unpruned': {'shap': [16.697126388549805,
   0.06618619710206985,
   0.47682851552963257,
   0.15279464423656464,
   25.01079750061035,
   0.16142603754997253]},
 'l1unstruct': {'shap': [16.697126388549805,
   0.06618619710206985,
   0.47682851552963257,
   0.15279464423656464,
   25.01079750061035,
   0.16142603754997253]},
 'randunstruct': {'shap': [16.697126388549805,
   0.06618619710206985,
   0.47682851552963257,
   0.15279464423656464,
   25.01079750061035,
   0.16142603754997253]}}